In [10]:
#!pwd
import pandas as pd
import os
import string
from nltk.corpus import stopwords
from nltk import word_tokenize, WordNetLemmatizer
from nltk import stem, pos_tag
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [11]:
df = pd.read_csv('/Users/Jill/Documents/New Media and Sentiment Analysis/CA/EB5204_Sentiment_Mining_CA/data/df_reviews_train.csv')
df.head()
ffile1 = open("/Users/Jill/Documents/New Media and Sentiment Analysis/CA/EB5204_Sentiment_Mining_CA/data/train.csv","r", encoding = "ISO-8859-1")
df_standard = pd.read_csv(ffile1, encoding = "utf-8")
df_standard.drop(['restaurant_id', 'date', 'review_id'], inplace=True, axis=1)
df_standard.rename(columns={'text': 'review', 'Sentiment': 'sentiment'}, inplace=True)
df_standard.head()

mask = df_standard.sentiment == 'negative'
column_name = 'sentiment'
df_standard.loc[mask, column_name] = -1
mask = df_standard.sentiment == 'positive'
column_name = 'sentiment'
df_standard.loc[mask, column_name] = 1
df_standard = df_standard.rename(columns = {'stars':'rating'})
df_standard.head()

df = pd.concat([df, df_standard], axis=0)
df = df.reset_index().drop('index', axis=1)

In [12]:
ratings = df['rating']
sentiments = df['sentiment']
reviews = df['review']

In [13]:
sentiment_trans = []
for r in ratings.keys():
    rating = ratings.loc[r]
    sentiment = sentiments.loc[r]
    sentiment = int(sentiment)
    if rating <= 3:
        sentiment = -1
    else:
        sentiment = sentiment
    sentiment_trans.insert(r, sentiment)
sentiments = pd.Series(sentiment_trans, index=ratings.keys())

df = pd.concat([ratings, reviews, sentiments], axis = 1)
df.columns = ['rating', 'review', 'sentiment']
df.head()

,rating,review,sentiment
0,5,In my younger days when lunch choices consiste...,1
1,3,"After going through yelp and tripadvisor, I wa...",-1
2,5,Ordered Sichuan Prawns and Singapore Rice Nood...,1
3,3,Wong Kei is one of the many options you'll fin...,-1
4,1,One of the worst experience in a restaurant in...,-1


In [14]:
stop = stopwords.words('english')
snowball = stem.snowball.EnglishStemmer()
wnl = WordNetLemmatizer()
reviews = df['review']

In [15]:
# Remove all the punctuations and numbers
def removePunc(reviews):
    comwoPunc = str.maketrans({key: None for key in string.punctuation
                               + string.digits})
    for i in reviews.keys():
        comment = reviews.loc[i]
        reviewswoPunc = reviews.replace(comment, comment.translate(comwoPunc))
    return reviewswoPunc

In [16]:
# Convert all characters to Lower case
def convToLow(words):
    reviewsToLow = words.apply(str.lower)
    return reviewsToLow

In [17]:
# Stopwords removal
def removeStop(text, stop):
    for i in text.keys():
        comment = text.loc[i]
        comment_nostop = " ".join(filter(lambda word: word not in stop,
                                     comment.split()))
        reviewswoStop = text.replace(comment, comment_nostop)
    return reviewswoStop

In [18]:
# Tokenization
def token(text):
    reviewsToken = text.apply(word_tokenize)
    return reviewsToken

In [19]:
# pos tagging
def posTag(words):
    reviews_pos = words.apply(pos_tag)
    reviews_wnpos = []
    for i in reviews_pos.keys():
        comment = reviews_pos.loc[i]
        comment_wnpos = []
        for t in comment:
            t = list(t)
            tag = t[1]
            if t[1].startswith('J'):
                t[1] = wn.ADJ
            elif t[1].startswith('V'):
                t[1] = wn.VERB
            elif t[1].startswith('N'):
                t[1] = wn.NOUN
            elif t[1].startswith('R'):
                t[1] = wn.ADV
            else:
                del t
                t = None
            if t is not None:
                comment_wnpos.append(t)
            else:
                pass
        reviews_wnpos.append(comment_wnpos)
    reviews_wnpos = pd.Series(reviews_wnpos, index=reviews_pos.keys())
    return reviews_wnpos

In [20]:
# Lemmatization
def lemma(text, wnl):
    reviews_lem = []
    for i in text.keys():
        comment = text.loc[i]
        comment_lem = []
        for t in comment:
            word = t[0]
            tag = t[1]
            t = wnl.lemmatize(word, pos=tag)
            comment_lem.append(t)
        reviews_lem.append(comment_lem)
    allReviews = []
    for j in reviews_lem:
        reviews = ' '.join(j)
        allReviews.append(reviews)
    reviewsLemma = pd.Series(allReviews, index=text.keys())
    return reviewsLemma

In [ ]:
if __name__  == '__main__':
    reviews = removePunc(reviews)
    reviews = convToLow(reviews)
    reviews = removeStop(reviews, stop)
    reviews = token(reviews)
    reviews = posTag(reviews)
    final_reviews = lemma(reviews,wnl)
    ratings = df['rating']
    sentiments = df['sentiment']
    df = pd.concat([ratings, final_reviews, sentiments], axis = 1)
    df.columns = ['rating', 'review', 'sentiment']
    df.to_csv('/Users/Jill/Documents/New Media and Sentiment Analysis/CA/EB5204_Sentiment_Mining_CA/data/df_preprocessing.csv', encoding='utf-8')